In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import warnings

warnings.filterwarnings("ignore")
train_dir = '/kaggle/input/mri-dataset-for-alzheimers/alzheimer dataset/train'
test_dir = '/kaggle/input/mri-dataset-for-alzheimers/alzheimer dataset/test'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.4,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    # horizontal_flip=True,
    rotation_range=20,
    fill_mode='nearest',
    validation_split=0.2
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)
val_test_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128,128),
    color_mode='grayscale',
    class_mode='categorical',
    subset='training'
)

val_generator = val_test_datagen.flow_from_directory(
    train_dir,
    target_size=(128,128),
    color_mode='grayscale',
    class_mode='categorical',
    subset='validation',
    shuffle=False,
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128,128),
    color_mode='grayscale',
    class_mode='categorical'
)

2026-02-09 09:48:13.146585: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770630493.337287      23 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770630493.390856      23 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770630493.848593      23 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770630493.848655      23 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770630493.848658      23 computation_placer.cc:177] computation placer alr

Found 3719 images belonging to 4 classes.
Found 928 images belonging to 4 classes.
Found 1813 images belonging to 4 classes.


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

CNN1 = Sequential()

CNN1.add(Conv2D(32, (3,3), activation='relu', input_shape=(128,128,1)))
CNN1.add(BatchNormalization())
CNN1.add(MaxPooling2D(2,2))

CNN1.add(Conv2D(64, (3,3), activation='relu'))
CNN1.add(BatchNormalization())
CNN1.add(MaxPooling2D(2,2))

CNN1.add(Conv2D(128, (3,3), activation='relu'))
CNN1.add(BatchNormalization())
CNN1.add(MaxPooling2D(2,2))

CNN1.add(Conv2D(256, (3,3), activation='relu'))
CNN1.add(BatchNormalization())
CNN1.add(MaxPooling2D(2,2))

CNN1.add(Flatten())

CNN1.add(Dense(32, activation='relu'))
CNN1.add(Dropout(0.2))

CNN1.add(Dense(32, activation='relu'))
# CNN1.add(Dropout(0.2))

CNN1.add(Dense(4, activation='softmax'))

# optimizer = Adam(learning_rate=1e-4)
CNN1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
CNN1.summary()

I0000 00:00:1770630508.181530      23 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15511 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 61, 61, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 12, 12, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 12, 12, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │       294,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 685,892 (2.62 MB)

 Trainable params: 684,932 (2.61 MB)

 Non-trainable params: 960 (3.75 KB)

In [4]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(
    monitor="val_loss",     # metric to monitor
    patience=6,             # epochs to wait before stopping
    restore_best_weights=True
)

In [5]:
history = CNN1.fit(train_generator, validation_data=val_generator, callbacks=[early_stop], epochs=100)

Epoch 1/100


I0000 00:00:1770630513.775983      89 service.cc:152] XLA service 0x7ac7e4015b40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1770630513.776032      89 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1770630514.508010      89 cuda_dnn.cc:529] Loaded cuDNN version 91002


  1/117 ━━━━━━━━━━━━━━━━━━━━ 17:54 9s/step - accuracy: 0.5312 - loss: 1.2340

I0000 00:00:1770630519.558877      89 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


117/117 ━━━━━━━━━━━━━━━━━━━━ 40s 262ms/step - accuracy: 0.5404 - loss: 1.3144 - val_accuracy: 0.2737 - val_loss: 1.5422
Epoch 2/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 10s 86ms/step - accuracy: 0.5702 - loss: 0.9889 - val_accuracy: 0.4375 - val_loss: 1.1462
Epoch 3/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 10s 85ms/step - accuracy: 0.5933 - loss: 0.9201 - val_accuracy: 0.5938 - val_loss: 0.9527
Epoch 4/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 10s 85ms/step - accuracy: 0.5790 - loss: 0.8914 - val_accuracy: 0.5765 - val_loss: 1.0816
Epoch 5/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 10s 86ms/step - accuracy: 0.5853 - loss: 0.8664 - val_accuracy: 0.5733 - val_loss: 0.9436
Epoch 6/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 10s 84ms/step - accuracy: 0.5794 - loss: 0.8647 - val_accuracy: 0.5571 - val_loss: 1.1872
Epoch 7/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 10s 83ms/step - accuracy: 0.5883 - loss: 0.8426 - val_accuracy: 0.5938 - val_loss: 1.2057
Epoch 8/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 10s 85ms/step - accuracy: 0.5889 - loss: 0.8423 - val